# **Assignment 2 - (Based on Data Collection using API)**

**By Siddhant - 102203023**

Importing Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

Importing the API Key

In [2]:
API_KEY = %run ./Alpha_Vantage_API_Key.ipynb

Adding the URL

In [3]:
url = 'https://www.alphavantage.co/query?apikey={API_KEY}'

Alpha Vantage is a widely used tool for accessing and analyzing financial market data. The Alpha Vantage Application Programming Interface (API) is a developer-friendly, easy-to-use interface to access global stock data. The API allows you to access live and historical stock data for various national and international assets. Using the following link, get the API key for the Alpha Vantage:

https://www.alphavantage.co/support/#api-key

Based on TimeSeries API:
1. Find the symbol (company codes) for the following companies using symbol search method of
the API:

    a. Apple

    b. Videocon

    c. Intel Corporation, USA

In [97]:
function ='SYMBOL_SEARCH'
def get_symbol(company):
    data = (requests.get(url+'&function='+function+'&keywords='+company)).json()
    return data.get('bestMatches', [])

In [98]:
companies = ["Apple", "Videocon", "Intel Corporation"] # Apple returns - Apple Hospitality, not Apple Inc
list = []
for company in companies:
    symbols = get_symbol(company)
    if symbols:
        list.append({'Company': company, 'Symbol (Company Code)': symbols[0]['1. symbol']})
        
df1 = pd.DataFrame(list)
df1 # Videocon is closed down now

,Company,Symbol (Company Code)
0,Apple,APLE
1,Videocon,511389.BSE
2,Intel Corporation,INL.DEX


2. Find the intraday data of Apple Corporation, United States after an interval of 15 mins in full
format (all the records available in the database).

In [100]:
function = 'TIME_SERIES_INTRADAY'
interval = '15min'
def intraday(company):
    response = requests.get(url+'&function='+function+'&interval='+interval+'&symbol=AAPL')
    data = response.json()
    return data
intraday('Apple')

{'Meta Data': {'1. Information': 'Intraday (15min) open, high, low, close prices and volume',
  '2. Symbol': 'AAPL',
  '3. Last Refreshed': '2024-08-30 19:45:00',
  '4. Interval': '15min',
  '5. Output Size': 'Compact',
  '6. Time Zone': 'US/Eastern'},
 'Time Series (15min)': {'2024-08-30 19:45:00': {'1. open': '229.1000',
   '2. high': '229.2200',
   '3. low': '229.0400',
   '4. close': '229.1500',
   '5. volume': '7852'},
  '2024-08-30 19:30:00': {'1. open': '229.1400',
   '2. high': '229.1400',
   '3. low': '229.0500',
   '4. close': '229.0600',
   '5. volume': '1963'},
  '2024-08-30 19:15:00': {'1. open': '229.1900',
   '2. high': '229.1900',
   '3. low': '229.0400',
   '4. close': '229.1000',
   '5. volume': '2190'},
  '2024-08-30 19:00:00': {'1. open': '229.1150',
   '2. high': '229.1900',
   '3. low': '229.0900',
   '4. close': '229.1900',
   '5. volume': '3835'},
  '2024-08-30 18:45:00': {'1. open': '229.0500',
   '2. high': '229.1500',
   '3. low': '229.0000',
   '4. close': '

3. Find the daily data of 100 entries of Videocon India

In [102]:
videocon_symbol = '511389.BSE'
function = 'TIME_SERIES_DAILY'
response = requests.get(url+'&function='+function+'&outputsize=compact&symbol='+videocon_symbol)
data = response.json()
if 'Time Series (Daily)' in data:
    daily_data = pd.DataFrame.from_dict(data['Time Series (Daily)'], orient='index')
    daily_data = daily_data.rename(columns={
        '1. open': 'Open',
        '2. high': 'High',
        '3. low': 'Low',
        '4. close': 'Close',
        '5. volume': 'Volume'
    })
    
    daily_data.index = pd.to_datetime(daily_data.index)
    daily_data = daily_data.sort_index(ascending=False)

    print(daily_data.head(100))

              Open    High     Low   Close Volume
2024-08-29  7.2500  7.2500  7.2500  7.2500      0
2024-08-28  7.2500  7.2500  7.2500  7.2500      0
2024-08-27  7.2500  7.2500  7.2500  7.2500      0
2024-08-26  7.2500  7.2500  7.2500  7.2500      0
2024-08-23  7.2500  7.2500  7.2500  7.2500      0
...            ...     ...     ...     ...    ...
2024-04-15  7.2500  7.2500  7.2500  7.2500      0
2024-04-12  7.2500  7.2500  7.2500  7.2500      0
2024-04-11  7.2500  7.2500  7.2500  7.2500      0
2024-04-10  7.2500  7.2500  7.2500  7.2500      0
2024-04-09  7.2500  7.2500  7.2500  7.2500      0

[100 rows x 5 columns]


Based on Alpha Intelligence API:

4. Find the title, summary, authors, overall_sentiment_score, and overall_sentiment_label of sentiment news for IBM on technology topic

In [7]:
function = 'NEWS_SENTIMENT'
tickers = 'IBM'
topics = 'technology'
response = requests.get(url+'&function='+function+'&tickers='+tickers+'&topics='+topics)
data = response.json()
#print(data)
if 'feed' in data:
    news=[]
    for x in data['feed']:
        news_data = {
            'Title': x['title'],
            'Summary': x['summary'],
            'Overall Sentiment Score': x['overall_sentiment_score'],
            'Overall Sentiment Label': x['overall_sentiment_label']
        }
        news.append(news_data)
    
    df_news = pd.DataFrame(news)
    print(df_news)
else: print('Error in retrieving data')

                                                Title  \
0   Should iShares MSCI USA Min Vol Factor ETF  ( ...   
1          IBM's Next Mainframe Will Pack an AI Punch   
2   Prediction: This Under-the-Radar Tech Stock Wi...   
3   Mobile Value-Added Services  ( MVaS )  Market ...   
4   IBM to Shut Down China R&D Operations: Will th...   
5   IBM Unveils Processor Innovations to Boost AI ...   
6   Wall Street's Most Accurate Analysts Give Thei...   
7   Zacks Industry Outlook IBM, Hewlett Packard En...   
8   Adobe Stock Plunges 6.4% YTD: Should You Buy t...   
9   Adobe  ( ADBE )  Stock Plunges 6.4% YTD: Shoul...   
10  3 Stocks to Watch From a Challenging Technolog...   
11  IBM Shuts Down China Operations, Shifts Resear...   
12  ServiceNow  ( NOW )  Down 1% Since Last Earnin...   
13  IBM  ( IBM )  Up 2.1% Since Last Earnings Repo...   
14  IBM Is Doing 'Great,' Cramer Says - Avoid This...   
15  2 Old-School Tech Giants Winning in the AI Market   
16  Should You Invest in the Fi

5. Find the top 20 gainers, losers, and the most active traded tickers in the US market.

In [10]:
function = 'TOP_GAINERS_LOSERS'
response = requests.get(url+'&function='+function)
data = response.json()

gainers_df = pd.DataFrame(data['top_gainers'])
print(gainers_df)

losers_df = pd.DataFrame(data['top_losers'])
print(losers_df)

most_active_df = pd.DataFrame(data['most_actively_traded'])
print(most_active_df)

   ticker   price change_amount change_percentage     volume
0   CEADW   0.014         0.008         133.3333%       4511
1    BNRG    1.55         0.877          130.312%  235766567
2   ACABW  0.0495        0.0274         123.9819%      66670
3   CYTHW   0.115         0.063         121.1538%        651
4   HYZNW   0.025        0.0124          98.4127%      22281
5    LGL+  0.3499        0.1699          94.3889%      34821
6   HNRA+  0.0793        0.0382           92.944%     149233
7   XBPEW  0.0358        0.0158             79.0%       3930
8   ZCARW  0.0233        0.0102          77.8626%       5967
9   RSVRW    1.34        0.5797          76.2462%      19830
10  AENTW  0.0418        0.0178          74.1667%      53932
11  LSBPW  0.0689        0.0289            72.25%       4300
12  CURIW   0.028        0.0109          63.7427%      74797
13  MCAGR  0.1003        0.0382          61.5137%      97884
14   ATPC    2.55         0.958          60.1759%   79522256
15  ETWO+    0.19       

Based on Fundamental Data
6. Find the annual and quarterly income statements for the company

In [11]:
function = 'INCOME_STATEMENT'
symbol = 'IBM'
response = requests.get(url+'&function='+function+'&symbol='+symbol)
data = response.json()
print(data)

{'symbol': 'IBM', 'annualReports': [{'fiscalDateEnding': '2023-12-31', 'reportedCurrency': 'USD', 'grossProfit': '34300000000', 'totalRevenue': '61860000000', 'costOfRevenue': '27560000000', 'costofGoodsAndServicesSold': '245000000', 'operatingIncome': '6979000000', 'sellingGeneralAndAdministrative': '19003000000', 'researchAndDevelopment': '6775000000', 'operatingExpenses': '27321000000', 'investmentIncomeNet': 'None', 'netInterestIncome': '-1607000000', 'interestIncome': '591000000', 'interestExpense': '1607000000', 'nonInterestIncome': '-77000000', 'otherNonOperatingIncome': '266000000', 'depreciation': '2109000000', 'depreciationAndAmortization': '2287000000', 'incomeBeforeTax': '8678000000', 'incomeTaxExpense': '1176000000', 'interestAndDebtExpense': '1607000000', 'netIncomeFromContinuingOperations': '7514000000', 'comprehensiveIncomeNetOfTax': '5481000000', 'ebit': '10285000000', 'ebitda': '12572000000', 'netIncome': '7502000000'}, {'fiscalDateEnding': '2022-12-31', 'reportedCurr

Based on Forex Rate
7. Find the real time exchange rate for USD to INR of digital currency

In [18]:
function = 'CURRENCY_EXCHANGE_RATE'
from_currency = 'USD'
to_currency = 'INR'
response = requests.get(url+'&function='+function+'&from_currency='+from_currency+'&to_currency='+to_currency)
data = response.json()
#print(data)
print(data['Realtime Currency Exchange Rate']['5. Exchange Rate'])

83.86000000
